In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

from data_dragon import DataDragon
from lol_rules import DRAGON_TYPES, TURRET_TYPES
from win_predictor.win_feature_extractor import WinFeatureExtractor
from win_predictor.logistic_regression_win_predictor import LogisticRegressionWinPredictor


In [2]:
data_folder = r'C:\Users\edems\Documents\Work\fow_AI\Game_data'
output_folder = os.path.join('output', 'win_prediction')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
output_folder = os.path.join(output_folder, 'win_visualization')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
dataset_folder = 'data_fow_v2'
version = '13.4'
data_dragon = DataDragon(data_folder, version)

df = pd.read_csv(os.path.join(dataset_folder, 'win_dataset.csv'))
game_ids = np.unique(df['id'])
kf = KFold(n_splits=5)




In [5]:
features = ['kills', 'turrets_total', 'monsters', 'gold']
for train_index, test_index in kf.split(game_ids):
    # Prepare train data
    rows = []
    for id in game_ids[train_index]:
        game_data = df.loc[df['id'] == id]
        times = [timestep for timestep in game_data['time']]
        for timestep in times:
            rows.extend(game_data.loc[game_data['time'] == timestep].index)
    x_train = df.iloc[rows]
    y_train = x_train.pop('winner').values
    x_train.pop('id')
    feature_extractor = WinFeatureExtractor(features,
                                            data_dragon,
                                            normalize=False)
    model = LogisticRegressionWinPredictor(feature_extractor)
    model.train(x_train, y_train)

C:\Users\edems\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\edems\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\edems\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\e

In [13]:
def plot_winrate(game_data: pd.DataFrame,
                 winner: int,
                 probabilities: np.ndarray,
                 width: int,
                 height: int,
                 vertical_padding: int,
                 horizontal_padding: int) -> np.ndarray:
    game_end = int(game_data.iloc[-1]['time'])
    game_end_minutes = game_end // 60 // 1000
    canvas = np.full((height, width, 3), 0, dtype=np.uint8)
    axes_color = (255, 255, 255)
    prediction_color = (255, 255, 255)
    text_font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    title_font_scale = 2
    tick_size = 10
    graph_height = height 
    # Add title
    title = "Win Prediction"
    cv2.putText(canvas, title, (width // 2 - len(title) * 15, vertical_padding - 10), text_font, title_font_scale, (255, 255, 255), 3)
    # Plot axes
    cv2.line(canvas, (horizontal_padding, vertical_padding), (horizontal_padding, graph_height - vertical_padding), axes_color, 2)
    cv2.line(canvas, (horizontal_padding, graph_height - vertical_padding), (width - horizontal_padding, graph_height - vertical_padding), axes_color, 2)
    cv2.line(canvas, (horizontal_padding, vertical_padding), (width - horizontal_padding, vertical_padding), axes_color, 2)
    
    # Add labels to axes
    cv2.putText(canvas, 'Minutes', (width - 115, height - 0), text_font, font_scale, axes_color, 2)

    
    # Add dots
    x_dot_num = np.ceil(game_end_minutes / 5).astype(int).item()
    x_per_dot = (width - 2 * horizontal_padding) / (x_dot_num - 1)
    y_axis_dots = np.linspace(graph_height - vertical_padding, vertical_padding, num=11, dtype=int)
    # Plot y axis ticks and labels
    for i in range(11):
        x = horizontal_padding
        y = y_axis_dots[i]
        label = f'{i / 10:.1f}'
        cv2.line(canvas,
                 (x, y),
                 (x - tick_size, y),
                 axes_color)
        cv2.putText(canvas,
                    label,
                    (x - 22 * len(label), y + 10 * font_scale),
                    text_font,
                    font_scale,
                    axes_color,
                    2)
    # Plot x axis ticks and labels
    for i in range(x_dot_num):
        x = horizontal_padding + round(i * x_per_dot)
        y = graph_height - vertical_padding
        label = str(i*5)
        cv2.line(canvas,
                 (x, y),
                 (x, y + tick_size),
                 axes_color)
        cv2.putText(canvas,
                    label,
                    (x - (20 * font_scale * len(label)) // 2, y + 40),
                    text_font,
                    font_scale,
                    axes_color,
                    2)
    # Plot predicted winrate and events
    previous_horizontal_offset = 0
    previous_vertical_offset = round((graph_height - 2 * vertical_padding) * probabilities[0, 0])
    for i, state_probabilities in enumerate(probabilities):
        horizontal_offset = round((width - 2 * horizontal_padding) * (i / len(game_data)))
        vertical_offset = round((graph_height - 2 * vertical_padding) * state_probabilities[0])
        previous_pos = (horizontal_padding + previous_horizontal_offset, graph_height - vertical_padding - previous_vertical_offset)
        pos = (horizontal_padding + horizontal_offset, graph_height - vertical_padding - vertical_offset)
        cv2.line(canvas, previous_pos, pos, prediction_color, 2)
        previous_horizontal_offset = horizontal_offset
        previous_vertical_offset = vertical_offset

    return canvas


width, height = 1500, 900
vertical_padding = 60
horizontal_padding = 70


id = game_ids[0]
x_test = df.loc[df['id'] == id]
y_test = x_test.pop('winner').values
x_test.pop('id')
probabilities = model.predict_proba(x_test)
img = plot_winrate(x_test, y_test[0], probabilities, width, height, vertical_padding, horizontal_padding)
cv2.imwrite(os.path.join(output_folder, f"win_prediction_{id}.png"), img)


True